# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846716198738                   -0.70    4.0
  2   -7.852287855800       -2.25       -1.53    1.0
  3   -7.852646004083       -3.45       -2.52    3.2
  4   -7.852646675803       -6.17       -3.33    2.5
  5   -7.852646685764       -8.00       -4.74    1.2
  6   -7.852646686726       -9.02       -5.19    4.0
  7   -7.852646686729      -11.50       -5.90    1.0
  8   -7.852646686730      -12.42       -7.34    2.0
  9   -7.852646686730      -14.15       -7.51    3.0
 10   -7.852646686730   +  -14.45       -8.25    2.0
 11   -7.852646686730      -14.45       -9.49    2.5
 12   -7.852646686730   +  -15.05      -10.60    3.0
 13   -7.852646686730   +  -15.05      -10.74    1.0
 14   -7.852646686730   +    -Inf      -11.42    2.0
 15   -7.852646686730      -15.05      -12.44    2.8


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846839209265                   -0.70           4.0
  2   -7.852526728262       -2.25       -1.62   0.80    4.8
  3   -7.852611236907       -4.07       -2.79   0.80    5.2
  4   -7.852646566453       -4.45       -3.55   0.80    7.2
  5   -7.852646683475       -6.93       -4.54   0.80    9.0
  6   -7.852646686678       -8.49       -5.08   0.80   10.5
  7   -7.852646686724      -10.33       -6.11   0.80   11.5
  8   -7.852646686730      -11.27       -7.16   0.80   13.2
  9   -7.852646686730      -13.35       -7.72   0.80   15.8
 10   -7.852646686730      -14.35       -8.81   0.80   16.5
 11   -7.852646686730   +    -Inf       -9.43   0.80   18.2
 12   -7.852646686730   +  -15.05      -10.27   0.80   19.5
 13   -7.852646686730   +    -Inf      -11.05   0.80   20.8
 14   -7.852646686730      -14.27      -11.91   0.80   22.5
 15   -7.852646686730   +  -14.27      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.371468e+01     2.945453e+00
 * time: 0.4269258975982666
     1     1.228237e+00     2.266776e+00
 * time: 0.6183168888092041
     2    -1.152611e+00     1.988337e+00
 * time: 0.6416358947753906
     3    -3.616653e+00     2.031923e+00
 * time: 0.6751389503479004
     4    -4.679412e+00     1.855589e+00
 * time: 0.7087409496307373
     5    -6.637113e+00     1.069465e+00
 * time: 0.7424159049987793
     6    -7.309482e+00     7.576629e-01
 * time: 0.7761390209197998
     7    -7.598966e+00     6.631159e-01
 * time: 0.7994189262390137
     8    -7.696940e+00     3.810115e-01
 * time: 0.822822093963623
     9    -7.748602e+00     1.718573e-01
 * time: 0.8463029861450195
    10    -7.778080e+00     1.045302e-01
 * time: 0.8699469566345215
    11    -7.809591e+00     9.753365e-02
 * time: 0.8933629989624023
    12    -7.826396e+00     7.151929e-02
 * time: 0.9168469905853271
    13    -7.838885e+00     3.933726e-02
 * time: 1.01135897636

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846789357228                   -0.70    4.0
  2   -7.852309209841       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686695                   -2.54
  2   -7.852646686730      -10.46       -5.81
  3   -7.852646686730   +  -14.75      -12.43
|ρ_newton - ρ_scf|  = 3.858662320147633e-13
|ρ_newton - ρ_scfv| = 3.687851165887306e-14
|ρ_newton - ρ_dm|   = 3.979947901256173e-10
